<a href="https://colab.research.google.com/github/diegovrosales/SP1_Laboratorios/blob/master/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout, Conv1D, MaxPooling1D, GRU

import numpy as np
import matplotlib.pylab as plt

#dark mode
plt.rc_context({'xtick.color':'w', 'ytick.color':'w', 'text.color':'w', 'axes.labelcolor':'w'})

seed=1234
np.random.seed(seed)
tf.random.set_seed(seed)

In [14]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(None,), dtype=tf.int64, encoder=<SubwordTextEncoder vocab_size=8185>),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Pot

In [15]:
encoder = info.features['text'].encoder
print ('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [16]:
sample_input = 'Hello, world!'
encoded = encoder.encode(sample_input)
decoded = encoder.decode(encoded)
print(encoded, decoded, "\n")

for index in encoded:
  print('{}\t-> {}'.format(index, encoder.decode([index])))

[4025, 8040, 2, 562, 7962] Hello, world! 

4025	-> Hell
8040	-> o
2	-> , 
562	-> world
7962	-> !


In [0]:
BATCH_SIZE = 64
BUFFER_SIZE = 100000

In [18]:
train_shape = tuple(map(lambda x:x.shape,train_dataset.element_spec))
test_shape = tuple(map(lambda x:x.shape,test_dataset.element_spec))
train_shape, test_shape

((TensorShape([None]), TensorShape([])),
 (TensorShape([None]), TensorShape([])))

In [19]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_shape)
test_dataset = test_dataset.padded_batch(BATCH_SIZE,test_shape)

train_dataset.element_spec

(TensorSpec(shape=(None, None), dtype=tf.int64, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [0]:
EPOCHS = 5
STEPS_PER_EPOCH = 32
VALIDATION_STEPS = 16

In [0]:
def train(model, epochs=EPOCHS):
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

  return model.fit(train_dataset, epochs=epochs,
                      steps_per_epoch=STEPS_PER_EPOCH,
                      validation_data=test_dataset,
                      validation_steps=VALIDATION_STEPS
                      )

In [0]:
def plot_accuracy_and_loss(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  plt.figure(figsize=(10, 10))
  plt.subplot(2, 1, 1)
  plt.plot(acc, label='Training Accuracy')
  plt.plot(val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.setp(plt.legend().get_texts(), color='black')
  plt.ylabel('Accuracy')
  plt.ylim([min(plt.ylim()),1.0])
  plt.title('Training and Validation Accuracy')

  plt.subplot(2, 1, 2)
  plt.plot(loss, label='Training Loss')
  plt.plot(val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.setp(plt.legend().get_texts(), color='black')
  plt.ylabel('Cross Entropy')
  plt.ylim([0.0,1.0])
  plt.title('Training and Validation Loss')
  plt.xlabel('epoch')
  plt.show()

In [23]:
model = Sequential([
    Embedding(encoder.vocab_size, 32, mask_zero=True),
    Bidirectional(LSTM(64)),
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          261920    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               49664     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 311,713
Trainable params: 311,713
Non-trainable params: 0
_________________________________________________________________


In [0]:
history = train(model)
plot_accuracy_and_loss(history)

Train for 32 steps, validate for 16 steps
Epoch 1/5
32/32 [==============================] - 150s 5s/step - loss: 0.6923 - accuracy: 0.5381 - val_loss: 0.6895 - val_accuracy: 0.5918
Epoch 2/5
32/32 [==============================] - 153s 5s/step - loss: 0.6639 - accuracy: 0.6265 - val_loss: 0.6200 - val_accuracy: 0.7100
Epoch 3/5
32/32 [==============================] - 142s 4s/step - loss: 0.5553 - accuracy: 0.7471 - val_loss: 0.5447 - val_accuracy: 0.7490
Epoch 4/5
32/32 [==============================] - 156s 5s/step - loss: 0.5057 - accuracy: 0.7886 - val_loss: 0.4966 - val_accuracy: 0.7959
Epoch 5/5
15/32 [=============>................] - ETA: 1:10 - loss: 0.4791 - accuracy: 0.8000

In [0]:
model = Sequential([
    Embedding(encoder.vocab_size, 32, mask_zero=True),
    Bidirectional(LSTM(64,  return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(1, activation='sigmoid')
])

model.summary()